# LSTM (Long Short Term Memory)

There is a branch of Deep Learning that is dedicated to processing time series. These deep Nets are **Recursive Neural Nets (RNNs)**. LSTMs are one of the few types of RNNs that are available. Gated Recurent Units (GRUs) are the other type of popular RNNs.

This is an illustration from http://colah.github.io/posts/2015-08-Understanding-LSTMs/ (A highly recommended read)

![RNNs](./images/RNN-unrolled.png)

Pros:
- Really powerful pattern recognition system for time series

Cons:
- Cannot deal with missing time steps.
- Time steps must be discretised and not continuous.

Also read [The Unreasonable Effectiveness of RNNs](karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy. Finish with having a browse through this [Stackoverflow Question](https://stackoverflow.com/questions/38714959/understanding-keras-lstms).

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, LSTM, Embedding, TimeDistributed

Using TensorFlow backend.


In [2]:
def chr2val(ch):
    ch = ch.lower()
    if ch.isalpha():
        return 1 + (ord(ch) - ord('a'))
    else:
        return 0
    
def val2chr(v):
    if v == 0:
        return ' '
    else:
        return chr(ord('a') + v - 1)

In [4]:
with open("data/sonnets.txt") as f:
# with open("data/AliceWonderland.txt") as f:
    text = f.read()
    
text_num = np.array([chr2val(c) for c in text])
print(text[:100])
print(text_num[:100])

THE SONNETS
by William Shakespeare




I

From fairest creatures we desire increase,
That thereby be
[20  8  5  0 19 15 14 14  5 20 19  0  2 25  0 23  9 12 12  9  1 13  0 19  8
  1 11  5 19 16  5  1 18  5  0  0  0  0  0  9  0  0  6 18 15 13  0  6  1  9
 18  5 19 20  0  3 18  5  1 20 21 18  5 19  0 23  5  0  4  5 19  9 18  5  0
  9 14  3 18  5  1 19  5  0  0 20  8  1 20  0 20  8  5 18  5  2 25  0  2  5]


The range of numbers for the letters are between:

In [5]:
[min(text_num), max(text_num)]

[0, 26]

Prepare the data

In [6]:
len_vocab = 27
sentence_len = 40
# n_chars = len(text_num)//sentence_len*sentence_len
num_chunks = len(text_num)-sentence_len

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    slice_size = batch_size * seq_length
    n_batches = len(int_text) // slice_size
    x = int_text[: n_batches*slice_size]
    y = int_text[1: n_batches*slice_size + 1]

    x = np.split(np.reshape(x,(batch_size,-1)),n_batches,1)
    y = np.split(np.reshape(y,(batch_size,-1)),n_batches,1)
    return x, y

x = np.zeros((num_chunks, sentence_len))
y = np.zeros(num_chunks)
for i in range(num_chunks):
    x[i,:] = text_num[i:i+sentence_len]
    y[i] = text_num[i+sentence_len]

# x = np.reshape(x, (num_chunks, sentence_len, 1))

In [7]:
x.shape

(95610, 40)

## Many to One Model

### Given fourty letters/charaters, predict next one letter/charater, with moving window size of 'fourty letters'.

In [8]:
model = Sequential()
model.add(Embedding(len_vocab, 64))
model.add(LSTM(64))
model.add(Dense(len_vocab, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          1728      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 27)                1755      
Total params: 36,507
Trainable params: 36,507
Non-trainable params: 0
_________________________________________________________________


In [9]:
Embedding?

In [10]:
np.random.choice(3,10,p=[0.99, 0.01, 0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
for i in range(10):
    model.fit(x,y, batch_size=128, epochs=1)
    sentence = []
    idx = np.random.choice(len(x),1)
    x_test = x[idx]
    if idx==len(x)-1:
        idx -= 1
#     sentence.append(val2chr(idx[0]))
    for i in range(100): # To predict 100 letters
        p = model.predict(x_test)
        idx2 = np.random.choice(27,1,p=p.ravel())
        x_test = np.hstack([x_test[:,1:], idx2[None,:]])
        sentence.append(val2chr(idx2[0]))

    print(''.join(sentence))
    print('-'*20)
    print(''.join([val2chr(int(v)) for v in x[idx+1,:].tolist()[0]]))
    print('='*40)

Epoch 1/1
95610/95610 [==============================] - 87s 906us/step - loss: 2.4165
  xs qyo mers prlse finl fodd denof went ne suls ant note heilir aln me ofy thii freane grosust so w
--------------------
h s conquest and make worms thine heir  
Epoch 1/1
95610/95610 [==============================] - 99s 1ms/step - loss: 2.0494
 my anles no winter foulln wee wit thout thee ikl  palist tel autr thouth haghint im   my argord tot
--------------------
and still weep that thou no form of thee
Epoch 1/1
95610/95610 [==============================] - 99s 1ms/step - loss: 1.9134
nd tore    in trou gait  you hen thar ire s aal stolld    reesd imfey d wraightn asceiw ly mate wise
--------------------
calls  it fears not policy  that heretic
Epoch 1/1
95610/95610 [==============================] - 101s 1ms/step - loss: 1.8335
swailt loves crong hisprouts ill dsroving thite fupcer eye howhpave s leming ill sworl pold boves i 
--------------------
els  i return again  just to the time  n
Epoch

In [12]:
idx2.shape

(1,)

In [13]:
p

array([[  6.25905595e-05,   6.29131272e-02,   2.65450562e-05,
          5.20197063e-05,   1.95484638e-04,   1.90559417e-01,
          5.17185370e-04,   8.71380144e-06,   4.72584034e-05,
          3.49257350e-01,   1.39614349e-05,   2.25702606e-05,
          3.73238837e-03,   7.11651228e-05,   6.45952023e-05,
          3.67691398e-01,   6.50743314e-05,   2.71877252e-05,
          2.61324545e-04,   2.00283012e-05,   1.08672000e-04,
          1.68368348e-03,   3.23718414e-03,   3.11368640e-04,
          3.92752438e-04,   1.86543800e-02,   2.56498925e-06]], dtype=float32)

In [14]:
sum(p.ravel())

0.99999999040073817

## Many to Many LSTM

### Given one letter/charater, predict next one letter/charater; Then use current predicted letter as input for next letter prediction.

In the previous layer we predicted one time step given the last 40 steps. This time however, we are predicting the 2nd to 41st character given the first 40 characters. Another way of looking at this is that, at each **character input** we are predicting the subsequent character.

In [15]:
len_vocab = 27
sentence_len = 40
# n_chars = len(text_num)//sentence_len*sentence_len
num_chunks = len(text_num)-sentence_len

x = np.zeros((num_chunks, sentence_len))
y = np.zeros((num_chunks, sentence_len))
for i in range(num_chunks):
    x[i,:] = text_num[i:i+sentence_len]
    y[i,:] = text_num[i+1:i+sentence_len+1]
y = y.reshape(y.shape+(1,))

In [16]:
# batch_size = 64

model = Sequential()
model.add(Embedding(len_vocab, 64)) # , batch_size=batch_size
model.add(LSTM(256, return_sequences=True)) # , stateful=True
model.add(TimeDistributed(Dense(len_vocab, activation='softmax')))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          1728      
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 256)         328704    
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 27)          6939      
Total params: 337,371
Trainable params: 337,371
Non-trainable params: 0
_________________________________________________________________


In [17]:
for i in range(10):
    sentence = []
    letter = [np.random.choice(len_vocab,1)[0]] #choose a random letter
    for i in range(100): # To predict 100 letters
        sentence.append(val2chr(letter[-1]))
        p = model.predict(np.array(letter)[None,:])
        letter.append(np.random.choice(27,1,p=p[0][-1])[0])
    print(''.join(sentence))
    print('='*100)
    model.fit(x,y, batch_size=128, epochs=1)

kmluzvfpjylafovififgfphanvboohpfbs wzkiipjftdozqgawnlmjiqdmtadyhcjfenotqdwpclvskfltrssnfypbi bjgocfi
Epoch 1/1
95610/95610 [==============================] - 434s 5ms/step - loss: 2.0616
  for thee  hy stould sour decor write me sonos dand lovy thing ast look how  outhinds gail d abbist
Epoch 1/1
95610/95610 [==============================] - 624s 7ms/step - loss: 1.4566
parts no show    in thee  distill celld fight  d fors the words s onguch sh   cvil  i fauth i noty  
Epoch 1/1
95610/95610 [==============================] - 412s 4ms/step - loss: 1.1793
words of love  thy book for my fair what nines seem nature rank edoth doth death   thus flattere rig
Epoch 1/1
95610/95610 [==============================] - 395s 4ms/step - loss: 0.9542
umper is took  how what it then in hath new my music  as thou shalt haffe  o this  and their groan f
Epoch 1/1
95610/95610 [==============================] - 400s 4ms/step - loss: 0.7920
le and inferser self more decoment  when i have gone decrainating

In [18]:
# sentence = [] # To predict new sentence from random starting letter, un-comment to initialize prediciton result
letter = [np.random.choice(len_vocab,1)[0]] #choose a random letter
for i in range(100): # To predict 100 letters
    sentence.append(val2chr(letter[-1]))
    p = model.predict(np.array(letter)[None,:])
    letter.append(np.random.choice(27,1,p=p[0][-1])[0])
print(''.join(sentence))
print('='*100)

pention quite  durch of mire their dear delight   xxxvii  what you works to wrong   lxxi  thy sour lmerity  and purgetoe sope  that thou in thich gor new  swear to these boas  thou upon thy servant th


### Notes:
1. The shape of `y` is now the same as x, as we are not predicting just one character any more.
2. In the following cell, it is important to notice that I did not need to use a 40 length character as an input to the predictions. See below:

In [19]:
x.shape

(95610, 40)

---

In [29]:
sentence = []
letter = [np.random.choice(len_vocab,1)[0]] #choose a random letter
for i in range(100): # To predict 100 letters
    sentence.append(val2chr(letter[-1]))
    p = model.predict(np.array(letter)[None,:])
    letter.append(np.random.choice(27,1,p=p[0][-1])[0])
print(''.join(sentence))
print('='*100)

me painted  but  thou give not worth  whose his truth  or gave thee wit  or graves a separable spite
